In [144]:
# Setup the JupyterDash is deprecated, so we will use Dash instead
from dash import Dash, dcc, html, dash_table

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
import plotly.express as px
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

import pymongo
from pymongo import MongoClient

In [145]:
###########################
# Data Manipulation / Model
###########################
from edit_data import DataBase
client = MongoClient()
connection_string = 'mongodb://localhost:27017/db.perfumania_fragrances_prices_averaged'
DB = 'db'
COL = 'perfumania_fragrances'

In [146]:
# Connect to database via CRUD Module
db = DataBase()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

#Dictionary containing all unique product categories
brand_categories = dict(zip(df['Brand'].unique(), df['Brand'].unique()))
brand_categories[' All'] = ' All'

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

In [147]:
#########################
# Dashboard Layout / View
#########################
app = Dash(__name__)

#Add image at top of page
image_filename = "logo3.png"
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#Add some color
page_color = '#d4bcc6'
chart_color = '#c89bad'

app.layout = html.Div([
    dcc.Store(id='filtered-data'),
    html.Div([
        html.Header([
            html.Center([
                html.B(html.H1('Perfumania Fragrances'),
                       style={'backgroundColor': page_color,
                              'fontFamily' : 'Garamond (serif)', 'font-size':'30px'}),
                html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), width='200', height='200',
                         style={'vertical-align': 'middle'})
                ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'center',
                         'backgroundColor': page_color}),
     #Display filtering options (Dropdown bars)
            html.Div(className='row',
                    style={'display': 'flex', 'backgroundColor': page_color},
                    children=[
                        html.Div(
                            dcc.Dropdown(
                                id='brand-filter',
                                options=sorted(brand_categories),
                                placeholder= 'Filter By Brand',
                                maxHeight = 400
                            ),
                            style={'width': '20%'}
                        ),
                        html.Div(
                            dcc.Dropdown(
                                id='price-filter',
                                options = [
                                    {'label': 'All', 'value': 'All'},
                                    {'label': 'Under $50', 'value': 'under50'},
                                    {'label': '$50-$100', 'value': '50to100'},
                                    {'label': '$100-$200', 'value': '100to200'},
                                    {'label': '$200-$300', 'value': '200to300'},
                                    {'label': 'Over $300', 'value': 'over300'}
                                ],
                                placeholder = 'Filter By Price Range'
                                ),
                            style={'width': '20%'}
                        ),
                    ])
        ]),

    html.Div(className='row',
             style={'display':'flex'},
             children=[
                 html.Div(
                     dash_table.DataTable(
                            id='dataTable-id',
                            columns=[
                                {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
                            ],
                            data=df.to_dict('records'),
                            editable=False,
                            filter_action="native",
                            sort_action="native",
                            sort_mode="multi",
                            column_selectable="single",
                            row_selectable="single",
                            row_deletable=False,
                            selected_columns=[],
                            selected_rows=[0],
                            page_action="native",
                            page_current=0,
                            page_size=30,
                            style_header={
                                'backgroundColor': 'black',
                                'color': 'white'
                            },
                            style_data={
                                'backgroundColor': chart_color,
                                'color': 'white',
                                'whiteSpace': 'normal',
                                'height': 'auto'
                            },
                            style_cell={
                                'font_size': '25px'
                            }
                        ), style={'width':'60%', 'backgroundColor' : page_color}),
                        html.Div(
                            id='pieChart-id',
                            className = 'col s12 m6',
                            style={'width': '40%'}
                        )
             ]),
    html.Br()
    ])
])

In [148]:
#Callback for keeping filtered and stored data updated
@app.callback(
    Output('dataTable-id', 'data'),
    Input('filtered-data', 'data')
)
def update_data(data):
    if data is None:
        return df.to_dict('records')  
    return data
# Callback for updating the filtered data to dashboard
@app.callback(
    Output('filtered-data', 'data'),
    [Input('brand-filter', 'value'),
     Input('price-filter', 'value')]
)
def update_dashboard(brand_filter, price_filter):
    if brand_filter != ' All' and brand_filter is not None:
        if price_filter != 'All' and price_filter is not None:
            if price_filter == 'under50':
                df = pd.DataFrame.from_records(db.read({"Brand": brand_filter, 'Price': {'$lt': 50}}))
            elif price_filter == '50to100':
                df = pd.DataFrame.from_records(db.read({"Brand": brand_filter, 'Price': {'$gte': 50, '$lt': 100}}))
            elif price_filter == '100to200':
                df = pd.DataFrame.from_records(db.read({"Brand": brand_filter, 'Price': {'$gte': 100, '$lt': 200}}))
            elif price_filter == '200to300':
                df = pd.DataFrame.from_records(db.read({"Brand": brand_filter, 'Price': {'$gte': 200, '$lt': 300}}))
            elif price_filter == 'over300':
                df = pd.DataFrame.from_records(db.read({"Brand": brand_filter, 'Price': {'$gte': 300}}))
        else:
            df = pd.DataFrame.from_records(db.read({"Brand": brand_filter}))
    elif price_filter != 'All' and price_filter is not None:
        if price_filter == 'under50':
            df = pd.DataFrame.from_records(db.read({'Price': {'$lt': 50}}))
        elif price_filter == '50to100':
            df = pd.DataFrame.from_records(db.read({'Price': {'$gte': 50, '$lt': 100}}))
        elif price_filter == '100to200':
            df = pd.DataFrame.from_records(db.read({'Price': {'$gte': 100, '$lt': 200}}))
        elif price_filter == '200to300':
            df = pd.DataFrame.from_records(db.read({'Price': {'$gte': 200, '$lt': 300}}))
        elif price_filter == 'over300':
            df = pd.DataFrame.from_records(db.read({'Price': {'$gte': 300}}))
    else:
        df = pd.DataFrame.from_records(db.read({}))

    results = df.to_dict('records')
    if results:
        df.drop(columns=['_id'], inplace=True)
        return df.to_dict('records')
    else:
        return []

In [149]:
#Callbacks for pie chart
@app.callback(
    Output('pieChart-id', "children"),
    Input('filtered-data', "data")
)

def update_pie(viewData):
    if viewData is None:
            return
    df = pd.DataFrame.from_records(viewData)
    brandCounter = Counter(df['Brand']).most_common(10)
    top_brands = [brand[0] for brand in brandCounter]
    brandCount = [brand[1] for brand in brandCounter]
    #brandCount = df['Brand'].value_counts()
    return[
        dcc.Graph(
        figure=px.pie(df, values=brandCount,
                  names=top_brands, title= 'Top 10 Most Common Fragrance Brands'
                  ).update_layout(paper_bgcolor=page_color, autosize=False, width=1000, height=1200,
                                  font=dict(
                                        size=20
                                  ))
        )
    ]

In [150]:
app.run(debug=True)